# Импорты

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import json
import chromadb
from typing import List
from uuid import uuid4
from chromadb.config import Settings
from langchain_gigachat.embeddings import GigaChatEmbeddings

import dotenv
dotenv.load_dotenv(os.path.abspath(os.path.join('..', './.env')))

True

# GigaChatEmbeddings

In [2]:
GIGACHAT_API_KEY = os.environ["GIGACHAT_API_KEY"]

embedding_client = GigaChatEmbeddings(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False)

# Датасет

In [3]:
with open("./data/contextual_dataset_v2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [4]:
texts = []

for sample in dataset:
    texts.append(sample["description_chunk"])

print(len(texts))

12771


# Векторизация сэмплов

In [5]:
def get_embeddings(texts: List[str]) -> List[list]:
    """
    Получение эмбеддингов входных текстов.
    """
    embeddings = embedding_client.embed_documents(texts=texts)
    
    return embeddings

In [6]:
embeddings = get_embeddings(texts=texts)

# Создание ChromaDB + загрузка и сохранение данных

In [7]:
client = chromadb.PersistentClient(path='../src/chroma')

In [8]:
collection = client.get_or_create_collection(name="news_embeddings")

In [9]:
for sample, embedding in zip(dataset, embeddings):

    collection.add(
        documents=[sample["description_chunk"]],
        embeddings=[embedding],
        metadatas=[{"date": sample["date"], "topic": sample["topic"]}],
        ids=str(uuid4())
    )

# Тестирование работоспособности базы данных

In [10]:
import chromadb

CHROMA_DATA_PATH = "../src/chroma"
COLLECTION_NAME = "news_embeddings"
client = chromadb.PersistentClient(CHROMA_DATA_PATH)
collection = client.get_collection(COLLECTION_NAME)

In [11]:
text = "Почему нефть подорожала?"
embedding = get_embeddings([text])

In [12]:
queries = collection.query(
    query_embeddings=embedding,
    n_results=10
)

In [13]:
queries['documents']

[['Цены на нефть начали расти после вчерашнего снижения, что может быть связано с коррекцией рынка, изменениями в геополитической обстановке или ожиданиями сокращения поставок. Рост цен также может быть обусловлен восстановлением спроса на энергоносители.',
  'Цены на нефть демонстрируют рост в связи с увеличением спроса на энергоносители, а также из-за ограничений на добычу и поставки, введенных крупными производителями, такими как ОПЕК+. Это способствует укреплению цен на мировых рынках.',
  'Цены на нефть выросли на 4,5% менее чем за сутки из-за усиления геополитических рисков, включая напряженность на Ближнем Востоке и потенциальные угрозы перебоям в поставках. Это вызвало рост спроса на нефть как защитный актив.',
  'Цены на нефть демонстрируют рост, что может быть связано с рядом факторов, включая изменения в геополитической обстановке, колебания спроса на энергоносители, а также решения крупных производителей, таких как ОПЕК+, по регулированию объемов добычи.',
  'Цены на нефть 